## Импортируем pandas

In [ ]:
import pandas as pd

## Считываем данные с нужным разделителем

In [ ]:
df = pd.read_csv("bank-full.csv", sep=";")
df.head()

## Выбираем нужные столбцы

In [ ]:
columns = ["age", "job", "marital", "education", "balance", "housing","contact", "day",
           "month", "duration", "campaign", "pdays", "previous", "poutcome", "y"]

## Проверяем признаки на наличие пропущенных значений

In [ ]:
df_new = df[columns]
df_new.isnull().sum()

## Вопрос 1

**Находим самое частое значение для столбца `education`**

In [ ]:
df_new["education"].mode()

**Ответ: `secondary`**

## Вопрос 2
**Оцениваем признаки и их тип данных**

In [ ]:
df_new.info()

**Находим числовые признаки**

In [ ]:
int_features = df_new.select_dtypes(include="int64").columns.tolist()
int_features

**Строим `heatmap` на основе корреляционной матрицы**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

correlation_matrix = df_new[int_features].corr()

plt.figure(figsize=(8,6))
sns.heatmap(correlation_matrix, annot=True, cmap="PuBuGn")
plt.show()

**Проверяем, сколько уникальных значений принимает целевая переменная**

In [ ]:
df_new["y"].unique()

**Кодируем целевую переменную**

In [ ]:
df_new["y"] = df_new["y"].replace({"yes": 1, "no": 0})

**Разделяем данные на тренировочную, валидационную и тестовую выборки. Для начала отделим целевую переменную от остального датасета**

In [ ]:
from sklearn.model_selection import train_test_split

x = df_new.drop(["y"], axis=1)
y = df_new["y"]

**Разделяем на _temp_ и _test_**

In [ ]:
x_temp, x_test, y_temp, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42
)

**Разделяем на _train_ и _validation_**

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_temp, y_temp,
    test_size=0.2,
    random_state=42
)

**Ответ: наибольшая корреляция у `age` и `balance`**

## Вопрос 3
**Для расчёта взаимной информации находим категориальные признаки**

In [ ]:
from sklearn.feature_selection import mutual_info_classif

cat_features = df_new.select_dtypes(include="object").columns.tolist()
cat_features

**Делаем подсчет взаимной информации между `cat_features` и `y`**

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

x_train_encoded = x_train.copy()

for col in cat_features:
    le = LabelEncoder()
    x_train_encoded[col] = le.fit_transform(x_train_encoded[col])

score = mutual_info_classif(x_train_encoded[cat_features], y_train)
score
# np.round(score, 2)

**Визуализируем взаимную информацию с помощью `barplot`**

In [ ]:
result = pd.DataFrame({
    "Признак": cat_features,
    "Взаимная информация": np.round(score, 2)
}).sort_values("Взаимная информация", ascending=False)

plt.figure(figsize=(10,8))
sns.barplot(result, x="Взаимная информация", y="Признак", palette="viridis")
plt.xlabel("Взаимная информация")
plt.ylabel("Признак")
plt.show()

**Ответ: наибольшая взаимная информация наблюдается у переменной `poutcome`**

## Вопрос 4
**Закодируем категориальные признаки с помощью _one-hot_ кодирования**

In [ ]:
cat_cols = x_train.select_dtypes(include=["object"]).columns

x_train = pd.get_dummies(x_train, columns=cat_cols)
x_val = pd.get_dummies(x_val, columns=cat_cols)

**Проверяем, что в тренировочной и валидационной выборке одинаковые категориальные признаки**

In [ ]:
x_train, x_val = x_train.align(x_val, join="left", axis=1, fill_value=0)

**Обучаем логистическую регрессию на тренировочном наборе данных**

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(x_train, y_train)

**Рассчитываем точность на валидационном наборе данных и округляем её до 2-х знаков**

In [ ]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_val)
acc = np.round(accuracy_score(y_val, pred), 2)
print(acc)

**Ответ: получаем значение _accuracy_ = 0.7**

## Вопрос 5
**Для начала каждому признаку сопоставим название столбца/столбцов, связанных с ним в датасете**

In [ ]:
test_features = ["age", "balance", "marital", "previous"]

feature_columns = {}

for feature in test_features:
    if feature in x_train.columns:
        feature_columns[feature] = [feature]
    else:
        feature_columns[feature] = [col for col in x_train.columns if col.startswith(feature + "_")]

feature_columns

**Пользуясь техникой _feature elimination_ поочередно исключаем каждый признак и подсчитываем точность**

In [ ]:
diff = {}

for feature, cols in feature_columns.items():
    x_train_temp = x_train.drop(columns=cols)
    x_val_temp = x_val.drop(columns=cols)

    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(x_train_temp, y_train)
    pred = model.predict(x_val_temp)

    acc_new = accuracy_score(y_val, pred)

    diff[feature] = acc - acc_new

**Смотрим итоговый результат**

In [ ]:
diff

**Находим наименьшую разницу**

In [ ]:
min(diff.items(), key=lambda x: x[1])

**Ответ: наименьшая разница у признака `marital` и её значение равно -0.07232513132430196**

## Вопрос 6
**Обучаем логистическую регрессию с регуляризацией. Без параметра `C = 0`, так как он равен обратному значению коэффициента  $С = \frac{1}{\lambda}$**

In [ ]:
# C_values = [0, 0.01, 0.1, 1, 10]
C_values = [0.01, 0.1, 1, 10]

res = {}

for C in C_values:
    model = LogisticRegression(solver="liblinear", C=C, max_iter=1000, random_state=42)
    model.fit(x_train, y_train)
    pred = model.predict(x_val)
    accuracy = np.round(accuracy_score(y_val, pred), 3)
    res[C] = accuracy

**После обучения модели со всеми признаками получаем точность для каждого предсказания**

In [ ]:
res

**Находим максимальную точность**

In [95]:
max(res.items(), key=lambda x: x[1])

(0.01, np.float64(0.824))

**Ответ: `C = 0.01` даёт лучшую точность 0.824**